In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [2]:
mnist = input_data.read_data_sets('/home/huseinzol05/Documents/MNIST/MNIST_data', one_hot = True)

Extracting /home/huseinzol05/Documents/MNIST/MNIST_data/train-images-idx3-ubyte.gz
Extracting /home/huseinzol05/Documents/MNIST/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /home/huseinzol05/Documents/MNIST/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /home/huseinzol05/Documents/MNIST/MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
class Model:
    
    def __init__(self, learning_rate, y_shape):
        self.X = tf.placeholder(tf.float32, (None, 28, 28, 1))
        self.Y = tf.placeholder(tf.float32, (None, y_shape))
        
        def convolutionize(x, conv_w, h = 1):
            return tf.nn.conv2d(input = x, filter = conv_w, strides = [1, h, h, 1], padding = 'SAME')

        def pooling(wx):
            return tf.nn.max_pool(wx, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
        
        w1 = tf.Variable(tf.random_normal([3, 3, 1, 16], stddev = 0.5))
        b1 = tf.Variable(tf.zeros(shape = [16]))
        w2 = tf.Variable(tf.random_normal([3, 3, 16, 8], stddev = 0.5))
        b2 = tf.Variable(tf.zeros(shape = [8]))
        w3 = tf.Variable(tf.random_normal([3, 3, 8, 8], stddev = 0.5))
        b3 = tf.Variable(tf.zeros(shape = [8]))
        w4 = tf.Variable(tf.random_normal([128, y_shape], stddev = 0.5))
        b4 = tf.Variable(tf.zeros(shape = [y_shape]))

        conv1 = pooling(tf.nn.sigmoid(convolutionize(self.X, w1) + b1))
        conv2 = pooling(tf.nn.sigmoid(convolutionize(conv1, w2) + b2))
        conv3 = pooling(tf.nn.sigmoid(convolutionize(conv2, w3) + b3))
        conv3 = tf.reshape(conv3, [-1, 128])
        self.logits = tf.matmul(conv3, w4) + b4
        
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        
        self.correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, "float"))

In [4]:
learning_rate = 0.01
sess = tf.InteractiveSession()
model = Model(learning_rate, mnist.train.labels.shape[1])
sess.run(tf.global_variables_initializer())

In [6]:
EPOCH, LOSS, ACC = [], [], []
BATCH_SIZE = 128

for i in xrange(10):
    last = time.time()
    EPOCH.append(i)
    TOTAL_LOSS, ACCURACY = 0, 0
    for n in xrange(0, (mnist.train.images.shape[0] // BATCH_SIZE) * BATCH_SIZE, BATCH_SIZE):
        batch_x = mnist.train.images[n: n + BATCH_SIZE, :].reshape((-1, 28, 28, 1))
        cost, _ = sess.run([model.cost, model.optimizer], feed_dict = {model.X : batch_x, model.Y : mnist.train.labels[n: n + BATCH_SIZE, :]})
        ACCURACY += sess.run(model.accuracy, feed_dict = {model.X : batch_x, model.Y : mnist.train.labels[n: n + BATCH_SIZE, :]})
        TOTAL_LOSS += cost
    
    TOTAL_LOSS /= (mnist.train.images.shape[0] // BATCH_SIZE)
    ACCURACY /= (mnist.train.images.shape[0] // BATCH_SIZE)
    LOSS.append(TOTAL_LOSS); ACC.append(ACCURACY)
    print 'epoch: ' + str(i + 1) + ', loss: ' + str(TOTAL_LOSS) + ', accuracy: ' + str(ACCURACY) + ', s / epoch: ' + str(time.time() - last)

epoch: 1, loss: 0.27670943084, accuracy: 0.920928030303, s / epoch: 27.144755125
epoch: 2, loss: 0.188248241533, accuracy: 0.946896853147, s / epoch: 27.0147771835
epoch: 3, loss: 0.152987092488, accuracy: 0.95541958042, s / epoch: 26.8241751194
epoch: 4, loss: 0.129784707049, accuracy: 0.9622122669, s / epoch: 26.8857750893
epoch: 5, loss: 0.114598895856, accuracy: 0.967202068765, s / epoch: 27.4256939888
epoch: 6, loss: 0.101240242707, accuracy: 0.970880681818, s / epoch: 28.2523770332
epoch: 7, loss: 0.0919746177223, accuracy: 0.973594114219, s / epoch: 28.2267827988
epoch: 8, loss: 0.085458196739, accuracy: 0.975979749417, s / epoch: 28.3221931458
epoch: 9, loss: 0.0804469554527, accuracy: 0.977236305361, s / epoch: 28.1964430809
epoch: 10, loss: 0.0763231400005, accuracy: 0.978511072261, s / epoch: 28.5655319691


In [10]:
from sklearn import metrics
testing_acc, logits = sess.run([model.accuracy, tf.cast(tf.argmax(model.logits, 1), tf.int32)], feed_dict = {model.X : mnist.test.images.reshape((-1, 28, 28, 1)), model.Y : mnist.test.labels})
print 'testing accuracy: ' + str(testing_acc)

testing accuracy: 0.9762
